In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
# warnings.filterwarnings('ignore')
!pip install torch
import torch
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install langchain-community
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

def start():
  from huggingface_hub import notebook_login
  notebook_login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.8 MB/s eta 0:00:00


In [2]:
def load_model(model_name):
  from langchain import HuggingFacePipeline
  from transformers import AutoTokenizer
  import transformers
  import torch

  # model = "meta-llama/Llama-2-7b-chat-hf"
  model = model_name

  tokenizer = AutoTokenizer.from_pretrained(model)

  global pipeline
  pipeline = transformers.pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      temperature=0.01, #MODfd

      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
  )

  global llm
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [3]:
def load_promt(prompt_text):
  from langchain import PromptTemplate,  LLMChain

  template = f"""
                {prompt_text}
            """

  global template_len
  template_len = len(template)

  prompt = PromptTemplate(template=template, input_variables=["text"])

  global llm_chain
  llm_chain = LLMChain(prompt=prompt, llm=llm)



In [4]:
def infer(input_text):
  response = llm_chain.run(input_text)
  print(response)
  return response

In [5]:
def upload_data(filename):
    try:
        global input_df
        # conditional to able to handle data from url
        if filename.startswith('http'):
          if filename.endswith(".tsv"):
            input_df = pd.read_csv(filename, sep='\t', header=None, names=['text', 'target'])
          if filename.endswith(".csv"):
            input_df = pd.read_csv(filename, sep=',', header=None, names=['text', 'target'])
        else:
          input_df = pd.read_csv(f'/content/{filename}', header=None, names=['text', 'target'])
        input_df.reset_index(drop=True, inplace=True)
        # input_df = input_df.drop([0])
        # input_df.reset_index(drop=True, inplace=True)
        print("File successfully loaded")
    except Exception as e:
        print(f"Error reading file: {e}")

In [6]:
def batch_infer(input_df):
  global result_df
  pred_list = []
  with tqdm(input_df.text, desc="Infering") as pbar:
    for idx, text_i in enumerate(pbar):
      response_i = llm_chain.run(text_i)
      print(f"{idx}) {response_i}\n{'= '*75}\n")
      pred_list.append(response_i)
      pbar.update(1)
    pred_df = pd.DataFrame({'pred':pred_list})
    result_df = pd.concat([input_df, pred_df], axis=1)
    return result_df

In [7]:
def post_process(df):
  sentiment_pred_list =  []

  global result_df
  with tqdm(df.values, desc="Processing") as pbar:
    for idx, row in enumerate(pbar):
      row_text, _, row_sentiment = row
      start_check = template_len + len(row_text)
      text_check = row_sentiment[start_check:]
      text_check = re.sub('  +', '', text_check)

      re_segment = re.search('positive|neutral|negative', text_check.lower())
      re_start, re_end = re_segment.span() if re_segment else (0, 0)
      text_check_sentiment = text_check[re_start:re_end].lower()
      sentiment_pred_list.append(text_check_sentiment)
      pbar.update(1)

  pred_df = pd.DataFrame({'post_processed_pred':sentiment_pred_list})
  result_df = pd.concat([df, pred_df], axis=1)
  return result_df


In [8]:
def test(df):
  df.post_processed_pred = df.post_processed_pred.map({'positive':1, 'neutral':0, 'negative':-1})
  print(df.post_processed_pred.value_counts())
  nan_pred_count = int(df.post_processed_pred.isna().sum())
  print(f"Null predictions:\t{nan_pred_count}")
  df.post_processed_pred = df.post_processed_pred.fillna(0)
  df.target = df.target.map({'positive':1, 'neutral':0, 'negative':-1})

  print(f"Macro f1 score:\t\t{f1_score(df.target, df.post_processed_pred, average='macro')}")

  confusion_matrix = metrics.confusion_matrix(df.target, df.post_processed_pred)

  cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = set(df.target.unique()).union(df.post_processed_pred.unique()))

  cm_display.plot()
  plt.show()



In [9]:
def download_pred(file_name):
  result_df.to_csv(file_name, index=True)
  # files.download(file_name)
  print("Please check the 'file' tab!")

In [10]:
# prompt: make a nb gui for the above functions. but sparete the start and load_model functions

import ipywidgets as widgets
from IPython.display import display, clear_output

# Define widgets for the GUI
model_name_input = widgets.Text(description='Model Name:')
prompt_input = widgets.Textarea(description='Prompt:')
input_text_input = widgets.Textarea(description='Input Text:')
input_df_upload = widgets.Textarea(description='File Name:')
output_area = widgets.Output()
file_name_input = widgets.Text(description='File Name:')
download_button = widgets.Button(description='Download Predictions')
download_output_area = widgets.Output()

# Define button for each function
start_button = widgets.Button(description='Start')
load_model_button = widgets.Button(description='Load Model')
load_prompt_button = widgets.Button(description='Load Prompt')
infer_button = widgets.Button(description='Infer')
upload_input_data_button = widgets.Button(description='Load File')
batch_infer_button = widgets.Button(description='Batch Infer')
post_process_button = widgets.Button(description='Post-Process')
test_button = widgets.Button(description='Test')

# Define functions to be called when buttons are clicked
def on_start_button_clicked(b):
  with output_area:
    clear_output()
    start()

def on_load_model_button_clicked(b):
  with output_area:
    clear_output()
    model_name = model_name_input.value
    load_model(model_name)

def on_load_prompt_button_clicked(b):
  with output_area:
    clear_output()
    prompt_text = prompt_input.value
    load_promt(prompt_text)

def on_infer_button_clicked(b):
  with output_area:
    clear_output()
    input_text = input_text_input.value
    infer(input_text)

def on_upload_input_data(b):
  with output_area:
    clear_output()
    if input_df_upload.value:
      file_name = input_df_upload.value
      upload_data(file_name)
    else:
      print('Please upload an input CSV file.')

def on_batch_infer_button_clicked(b):
  with output_area:
    clear_output()
    if input_df_upload.value:
      batch_infer(input_df)
    else:
      print('Please upload an input DataFrame.')

def on_post_process_button_clicked(b):
  with output_area:
    clear_output()
    if not result_df.empty:
      post_process(result_df)
    else:
      print('Please infer a DataFrame first.')

def on_test_button_clicked(b):
  with output_area:
    clear_output()
    test(result_df)

def on_download_button_clicked(b):
  with output_area:
    clear_output()
    if 'result_df' in globals():
      file_name = file_name_input.value
      if not file_name.endswith('.csv'):
        file_name += '.csv'
      download_pred(file_name)  # Download the file
    else:
      print("No predictions to download yet. Please run 'Batch Infer' first.")


# Assign button click events
start_button.on_click(on_start_button_clicked)
load_model_button.on_click(on_load_model_button_clicked)
load_prompt_button.on_click(on_load_prompt_button_clicked)
infer_button.on_click(on_infer_button_clicked)
upload_input_data_button.on_click(on_upload_input_data)
batch_infer_button.on_click(on_batch_infer_button_clicked)
post_process_button.on_click(on_post_process_button_clicked)
test_button.on_click(on_test_button_clicked)
download_button.on_click(on_download_button_clicked)

# Display the GUI
display(start_button)
display(model_name_input)
display(load_model_button)
display(prompt_input)
display(load_prompt_button)
display(input_text_input)
display(infer_button)
display(input_df_upload)
display(upload_input_data_button)
display(batch_infer_button)
display(post_process_button)
display(test_button)
display(file_name_input)
display(download_button)
display(download_output_area)
display(output_area)

Button(description='Start', style=ButtonStyle())

Text(value='', description='Model Name:')

Button(description='Load Model', style=ButtonStyle())

Textarea(value='', description='Prompt:')

Button(description='Load Prompt', style=ButtonStyle())

...

Button(description='Test', style=ButtonStyle())

Text(value='', description='File Name:')

Button(description='Download Predictions', style=ButtonStyle())

Output()

Output()

In [22]:
result_df = result_df2.copy()

In [18]:
result_df3 = result_df.copy()

In [20]:
result_df.head(20)

,text,target,pred,post_processed_pred
0,kemarin gue datang ke tempat makan baru yang a...,-1,\n output the sentiment of the ...,-1.0
1,kayak nya sih gue tidak akan mau balik lagi ke...,-1,\n output the sentiment of the ...,-1.0
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",-1,\n output the sentiment of the ...,-1.0
3,ini pertama kalinya gua ke bank buat ngurusin ...,-1,\n output the sentiment of the ...,-1.0
4,waktu sampai dengan gue pernah disuruh ibu lat...,-1,\n output the sentiment of the ...,-1.0
5,pelayanan di hotel salak bogor tidak sebagus y...,-1,\n output the sentiment of the ...,0.0
6,ada apa dengan young lex ? kenapa dia harus me...,-1,\n output the sentiment of the ...,0.0
7,"gue pesimis sama manusia , terutama dalam ngur...",-1,\n output the sentiment of the ...,-1.0
8,saya kecewa karena pengeditan biodata penumpan...,-1,\n output the sentiment of the ...,-1.0
9,saya hari ini melakukan pemesanan dan tiket sa...,-1,\n output the sentiment of the ...,-1.0
